In [2]:
# 데이터 저장
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import datetime

# 기초 데이터 

year = datetime.datetime.today().strftime('%Y')

xmlUrl = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.xml?key=fade36c00d1990c6aee207b95ad80621&openStartDt="+year+"&itemPerPage=100"

xmlData = urllib.request.urlopen(xmlUrl)

xmlSoup = BeautifulSoup(xmlData,"html.parser")

xml_r_DF = pd.DataFrame([])
tmp_xml_df = pd.DataFrame([])
tmp_xml_df = tmp_xml_df.append({'movieCd':"",'movieNm':"",'movieNmEn':"",'openDt':"",'directorNm':""},ignore_index=True)

tempSoup = xmlSoup.find_all('movie')
for i in range(len(tempSoup)) :
    tmp_xml_df.at[0,"movieCd"] = tempSoup[i].moviecd.text
    tmp_xml_df.at[0,"movieNm"] = tempSoup[i].movienm.text
    tmp_xml_df.at[0,"movieNmEn"] = tempSoup[i].movienmen.text
    tmp_xml_df.at[0,"openDt"] = tempSoup[i].opendt.text
    directors = ""
    if len(tempSoup[i].find_all('director')) == 1 :
        directors += tempSoup[i].directors.find('peoplenm').text
    elif len(tempSoup[i].find_all('director')) > 1 :
        directors += tempSoup[i].find_all('director')[0].peoplenm.text
        for j in range(1,len(tempSoup[i].find_all('director'))) :
            directors += ", "+tempSoup[i].find_all('director')[j].peoplenm.text
    
    tmp_xml_df.at[0,"directorNm"] = directors
    
    xml_r_DF = xml_r_DF.append(tmp_xml_df,ignore_index = True)
    
    for j in range(len(tmp_xml_df.columns)) :
        tmp_xml_df.iat[0,j] = ""
    
# index값을 movieCd로 변경
xml_r_DF = xml_r_DF.set_index('movieCd')

# year, month, day 추가
xml_r_DF['year'] = xml_r_DF['openDt'].apply(lambda x : x[:4])
xml_r_DF['month'] = xml_r_DF['openDt'].apply(lambda x : x[4:6])
xml_r_DF['day'] = xml_r_DF['openDt'].apply(lambda x : x[6:])

xmlInfoUrl = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.xml?key=fade36c00d1990c6aee207b95ad80621&movieCd='

# actor num, actor, casting
xmlType = np.array([])
xmlActorNum = np.array([])
xmlActorName = np.array([])
xmlActorCast =np.array([])

for i in xml_r_DF.index :
    xmlUpdateData = urllib.request.urlopen(xmlInfoUrl+i)
    xmlUpSoup = BeautifulSoup(xmlUpdateData,"html.parser")
    
    xmlType = np.append(xmlType, xmlUpSoup.find('typenm').text)
    xmlActorNum = np.append(xmlActorNum, len(xmlUpSoup.find_all('actor')))
    
    name = ""
    cast = ""
    
    if xmlUpSoup.find_all('actor') != [] :
        name += xmlUpSoup.find_all('actor')[0].peoplenm.text
        if xmlUpSoup.find_all('actor')[0].cast != [] :
            cast += xmlUpSoup.find_all('actor')[0].cast.text
        for j in range(1,len(xmlUpSoup.find_all('actor'))) :
            name += ", "+xmlUpSoup.find_all('actor')[j].peoplenm.text
            if xmlUpSoup.find_all('actor')[j].cast.text != "" :
                cast += ", "+xmlUpSoup.find_all('actor')[j].cast.text
    xmlActorName = np.append(xmlActorName, name)
    xmlActorCast = np.append(xmlActorCast, cast)
    
xml_r_DF['type'] = xmlType
xml_r_DF['actorNum'] = xmlActorNum.astype('int64')
xml_r_DF['actorName'] = xmlActorName
xml_r_DF['actorCast'] = xmlActorCast

In [3]:
# 출력확인
xml_r_DF

,directorNm,movieNm,movieNmEn,openDt,year,month,day,type,actorNum,actorName,actorCast
movieCd,,,,,,,,,,,
20180962,김용훈,지푸라기라도 잡고 싶은 짐승들,BEASTS CLAWING AT STRAWS,20200212,2020,02,12,장편,12,"전도연, 정우성, 배성우, 윤여정, 정만식, 진경, 신현빈, 김준한, 정가람, 박지...","연희, 태영, 중만, 순자, 박사장, 영선, 미란, 재훈, 진태, 붕어, 지배인, 메기"
20198676,셀린 시아마,타오르는 여인의 초상,Portrait of a Lady on Fire,20200116,2020,01,16,장편,2,"아델 에넬, 노에미 멜랑",
20182447,김광빈,클로젯,The Closet,20200205,2020,02,05,장편,3,"하정우, 김남길, 허율","상원, 경훈, 이나"
20190009,,카잔자키스,Kazantzakis,20200130,2020,01,30,장편,1,오디세즈 파파스필리오풀로스,
20190962,미카엘 헤르비그,벌룬,Balloon,20200116,2020,01,16,장편,4,"프리드리히 뮈케, 카롤리네 슈허, 데이빗 크로스, 토마스 크레취만",
...,...,...,...,...,...,...,...,...,...,...,...
20192387,,꽉찬 느낌이 좋은 새엄마,Secret relationship with mother-in-law. Invite...,20200103,2020,01,03,장편,0,,
20192335,,꽃미모 며느리에게 갑질,Yome Ryakudatsu,20200103,2020,01,03,장편,0,,
20186647,신해강,청춘빌라 살인사건,Dogs in the House,20200102,2020,01,02,장편,7,"김영호, 김정팔, 윤봉길, 강한샘, 김태정, 백인호, 정현호",
